## Loading in the data from [Warframe Items](https://github.com/WFCD/warframe-items)

This uses the top level `data` folder from the above repo.

In [2]:
import json

with open('./data/json/Primary.json') as json_file:
    data_primary = json.load(json_file)

with open('./data/json/Secondary.json') as json_file:
    data_secondary = json.load(json_file)
    
with open('./data/json/Melee.json') as json_file:
    data_melee = json.load(json_file)
    
with open('./data/json/Sentinels.json') as json_file:
    data_sentinels = json.load(json_file)
    
with open('./data/json/Warframes.json') as json_file:
    data_warframes = json.load(json_file)

In [2]:
with open('./data/json/Misc.json') as json_file:
    data_misc = json.load(json_file)
    
for item in data_misc:
    print(item["name"])

[Ph] Eidolon Trapping A
[Ph] Eidolon Trapping B
[Ph] Eidolon Trapping C
[Ph] Eidolon Trapping D
[Ph] Eidolon Trapping E
[Ph] Eidolon Trapping E
[Ph]Fragmented Seismic Data
[Ph]Old World Rusty Burst Pistol
[Ph]Old World Rusty Pistol
[Ph]Old World Rusty Revolver
[Ph]Old World Rusty Shotgun
[Ph]Old World Rusty Smg
[Ph]Old World Salvage
/Lotus/Language/Items/Clancreator
/Lotus/Language/Items/Limbocodename
/Lotus/Language/Items/Projectsinisterfragments
/Lotus/Language/Items/Relayreconstructionitemaname
/Lotus/Language/Items/Relayreconstructionitembname
/Lotus/Language/Items/Relayreconstructionitemcname
/Lotus/Language/Shipfeatureitems/Shipfeaturealchemyroom
Additional Appearance Config Slot For This Item
Additional Mod Config Slot For This Item
Affinityboosterstoreitem
Alertfusionbundlelarge
Alertfusionbundlemedium
Alertfusionbundlesmall
Alertrewardfusionbundle
Alerts Segment
Alloy Plate
Alpha Corruptor
Alpine Monitor Sawgaw Tag
Amaryn'S Retreat Scene
Amethyst Nexifera Tag
Anomaly Shard
Ans

In [3]:
data_primary[0]["imageName"]

'acceltra.png'

In [4]:
for item in data_warframes[1]["components"]:
    print(item["name"])

Blueprint
Chassis
Neuroptics
Orokin Cell
Systems


In [ ]:
items = {}

def item_has_components(item):
    return "components" in item.keys()

def extract_useful_data_nested(data):
    """Extract the components needed for each items and store them against the item name in the items dictionary.
    Also, return the names of items added to the dictionary from the current set of provided data.
    """
    item_names = []
    for item in data:
        if item_has_components(item):
            item_components = []
            for component in item["components"]:
                if item_has_components(component):
                    # Is a part like chassis or wings, nested costs need to be accessed
                    for subcomponent in component["components"]:
                        item_components.append({"name": subcomponent["name"], "itemCount": subcomponent["itemCount"]})
                        item_components.append({"name": "Credits", "itemCount": component["buildPrice"]})
                else:
                    # Is probbaly orokin cells
                    item_components.append({"name": component["name"], "itemCount": component["itemCount"]})
                    item_components.append({"name": "Credits", "itemCount": item["buildPrice"]})
            
            items[item["name"]] = item_components
            
#             item_components = [{"name": component["name"], "itemCount": component["itemCount"]} for component in item["components"]]
#             item_components.append({"name": "Credits", "itemCount": item["buildPrice"]})
#             items[item["name"]] = item_components
#             item_names.append([item["name"], item["imageName"]])
    return item_names

# item_names_warframes = extract_useful_data_nested(data_warframes)

## Generating item-components.js, all-item-names.js, and category_data.pickle

In [4]:
import pickle

items = {}
images = []
safe_names = {}

def extract_useful_data(data):
    """Extract the components needed for each items and store them against the item name in the items dictionary.
    Also, return the names of items added to the dictionary from the current set of provided data.
    """
    item_names = []
    for item in data:
        if "components" in item.keys():
            item_components = [{"name": component["name"], "itemCount": component["itemCount"]} for component in item["components"]]
            item_components.append({"name": "Credits", "itemCount": item["buildPrice"]})
            items[item["name"]] = item_components
            images.append(item["imageName"])
            code_safe_name = item["name"].lower().replace(" ", "-")
            safe_names[item["name"]] = code_safe_name
            item_names.append([item["name"], item["imageName"], code_safe_name])
    return item_names

# Extract the costs from all of the loaded in data sets
item_names_primary = extract_useful_data(data_primary)
item_names_secondary = extract_useful_data(data_secondary)
item_names_melee = extract_useful_data(data_melee)
item_names_sentinels = extract_useful_data(data_sentinels)
       
print(item_names_primary)
print(item_names_secondary)
print(item_names_melee)
print(item_names_sentinels)

category_data = [
    {"name": "Primary", "item_info": item_names_primary},
    {"name": "Secondary", "item_info": item_names_secondary},
    {"name": "Melee", "item_info": item_names_melee},
    {"name": "Sentinels", "item_info": item_names_sentinels},
]

# Save the category data so that main.py can access it easily
pickle.dump( category_data, open("category_data.pickle", "wb"))

# Save the items dictionary as a javascript file, so that it can be directly loaded by the page
with open("./webpage/js/item-components.js", "w") as f:
    f.write("var itemComponents = " + json.dumps(items))
    
with open("./webpage/js/all-item-names.js", "w") as f:
    f.write("var allItemNames = " + json.dumps(safe_names))

[['Acceltra', 'acceltra.png', 'acceltra'], ['Amprex', 'amprex.png', 'amprex'], ['Arca Plasmor', 'arca-plasmor.png', 'arca-plasmor'], ['Argonak', 'argonak.png', 'argonak'], ['Astilla', 'astilla.png', 'astilla'], ['Attica', 'attica.png', 'attica'], ['Basmu', 'basmu.png', 'basmu'], ['Battacor', 'battacor.png', 'battacor'], ['Baza', 'baza.png', 'baza'], ['Baza Prime', 'baza-prime.png', 'baza-prime'], ['Boar', 'boar.png', 'boar'], ['Boar Prime', 'boar-prime.png', 'boar-prime'], ['Boltor', 'boltor.png', 'boltor'], ['Boltor Prime', 'boltor-prime.png', 'boltor-prime'], ['Braton Prime', 'braton-prime.png', 'braton-prime'], ['Braton Vandal', 'braton-vandal.png', 'braton-vandal'], ['Bubonico', 'bubonico.png', 'bubonico'], ['Burston', 'burston.png', 'burston'], ['Burston Prime', 'burston-prime.png', 'burston-prime'], ['Buzlok', 'buzlok.png', 'buzlok'], ['Cernos', 'cernos.png', 'cernos'], ['Cernos Prime', 'cernos-prime.png', 'cernos-prime'], ['Convectrix', 'convectrix.png', 'convectrix'], ['Corinth

In [41]:
from shutil import copyfile

for image in images:
    copyfile("data/img/" + image, "webpage/img/" + image)